In [32]:
from tensorflow.keras.datasets import mnist
(train_X, train_y), (test_X, test_y) = mnist.load_data()

train_X = train_X.reshape(-1, 28, 28, 1) / 255.0 
test_X = test_X.reshape(-1, 28, 28, 1) / 255.0 

print("train_X shape:", train_X.shape)
print("test_X shape:", test_X.shape)

train_X shape: (60000, 28, 28, 1)
test_X shape: (10000, 28, 28, 1)


In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), 
                 input_shape=(28,28,1), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))

model.add(Conv2D(64, kernel_size=(3,3), activation="relu"))
model.add(MaxPooling2D()) # pool_size=(2,2)는 기본값 
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation="softmax"))

In [34]:
from tensorflow.keras.callbacks import ModelCheckpoint
filename = "model-digits.h5" 
checkpoint = ModelCheckpoint(filepath=filename,
                             monitor="val_accuracy", 
                             save_best_only=True, verbose=1)

In [35]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor="val_accuracy",  
                               patience=5, verbose=1)

In [36]:
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer="adam",
              metrics=["accuracy"])

In [37]:
model.fit(train_X, train_y, validation_data=(test_X, test_y),
          callbacks=[checkpoint, early_stopping], 
          batch_size=128, epochs=10)

Epoch 1/10
469/469 [==============================] - ETA: 0s - loss: 0.3369 - accuracy: 0.8957
Epoch 00001: val_accuracy improved from -inf to 0.97680, saving model to model-digits.h5
469/469 [==============================] - 5s 3ms/step - loss: 0.3369 - accuracy: 0.8957 - val_loss: 0.0763 - val_accuracy: 0.9768
Epoch 2/10
464/469 [============================>.] - ETA: 0s - loss: 0.1142 - accuracy: 0.9664
Epoch 00002: val_accuracy improved from 0.97680 to 0.98540, saving model to model-digits.h5
469/469 [==============================] - 1s 3ms/step - loss: 0.1137 - accuracy: 0.9665 - val_loss: 0.0464 - val_accuracy: 0.9854
Epoch 3/10
453/469 [===========================>..] - ETA: 0s - loss: 0.0830 - accuracy: 0.9751
Epoch 00003: val_accuracy improved from 0.98540 to 0.98790, saving model to model-digits.h5
469/469 [==============================] - 1s 3ms/step - loss: 0.0826 - accuracy: 0.9754 - val_loss: 0.0395 - val_accuracy: 0.9879
Epoch 4/10
460/469 [==========================

In [38]:
model.evaluate(test_X, test_y)

313/313 [==============================] - 0s 1ms/step - loss: 0.0225 - accuracy: 0.9926


[0.022457359358668327, 0.9926000237464905]

In [40]:
import cv2
import numpy as np

cap = cv2.VideoCapture(1)

if cap.isOpened():
    while True:
        ret, img = cap.read()
        if ret:
            g_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            _, bin_img = cv2.threshold(g_img, 110, 255,                                        cv2.THRESH_BINARY_INV)
            contours, hierarchy = cv2.findContours(bin_img,                    cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            try:
                for contour in contours:
                    (x,y), radius = cv2.minEnclosingCircle(contour)
                    if radius > 5:
                        xs, xe = int(x-radius), int(x+radius)
                        ys, ye = int(y-radius), int(y+radius)
                        cv2.rectangle(bin_img, (xs,ys),                                      (xe,ye), (200,0,0), 1)
                        roi = bin_img[ys:ye, xs:xe]
                        dst = cv2.resize(roi, dsize=(50,50))
                        dst = cv2.resize(dst, dsize=(24,24))
                        A = np.zeros((28,28))
                        A[2:-2,2:-2] = dst[:,:]
                        A = A.reshape(-1,28,28,1)
                        num = np.argmax(model.predict(A))
                        cv2.putText(bin_img, str(num), (xs, ys),                            cv2.FONT_HERSHEY_PLAIN, 2, (200,0,0))
            except Exception as e:
                print(e)
            cv2.imshow("Camera", bin_img)
            if cv2.waitKey(1)&0xFF == 27: # ESC
                break
        else:
            print("No Frame")
            break
else :
    print("Camera not opened")
    
cap.release()
cv2.destroyAllWindows()

OpenCV(4.5.4-dev) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.4-dev) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.4-dev) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.4-dev) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.4-dev) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.4-dev) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.4-dev) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.4-dev) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.4-dev) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.4-dev) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.4-dev) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.4-dev) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'